In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.ipac.ned import Ned
import numpy as np
from astropy.cosmology import WMAP9 as cosmo
from urllib.request import urlretrieve
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from astroquery.esa.hubble import ESAHubble

def read_mul171():
    # read mullaney and add designations
    mul171_ = pd.read_pickle("/home/insepien/research-data/alpaka/mull171.pkl")
    designations = []
    for i in range(0, len(mul171_)):
        pos = SkyCoord(ra=mul171_['RA'][i]*u.deg, dec=mul171_['DEC'][i]*u.deg)
        posstring = pos.to_string('hmsdms').split(' ')
        des_ra = posstring[0][0:2]+posstring[0][3:5]
        des_dec = posstring[1][0:3]+posstring[1][4:6]
        des = 'J'+des_ra+des_dec
        designations.append(des)
    mul171_['DESIG'] = designations
    return mul171_

def pos(row):
    """make skyCoord object for HST coord cone search"""
    return SkyCoord(ra=row['RA']*u.deg, dec=row['DEC']*u.deg)

def cross_bigmac(mul171):
    """crossmatch sample with big MAC"""
    # read in big  mac
    bigmac = pd.read_csv("/home/insepien/research-data/GFG.csv")
    # format designation
    desigs = []
    for i in range(len(bigmac)):
        name = bigmac['Name1'].loc[i].replace("SDSS","")
        if name[0] == "J":
            if "+" in name:
                desig = name.split("+")[0][:5] + "+" + name.split("+")[1][:4]
                desigs.append(desig)
            elif "-" in name:
                desig = name.split("-")[0][:5] + "-" + name.split("-")[1][:4]
                desigs.append(desig)
            else: print(name) 
        else:
            desigs.append(name)
    bigmac['DESIG'] = desigs

    # merge big mac and mullaney
    mul_bm = pd.merge(mul171,bigmac, on="DESIG")

    # optionally can get decals images
    # for n in mul_bm.index:
    #     urlretrieve('http://legacysurvey.org/viewer/jpeg-cutout?ra='+str(mul_bm.loc[n,'RA'])+'&dec='+str(mul_bm.loc[n,'DEC'])+'&layer=decals-dr7&pixscale=0.27&bands=grz',
    #                 "/home/insepien/research-data/hst/mul_bm/"+str(mul_bm.loc[n,'DESIG'])+'.jpg')
    return mul_bm, bigmac

def cal_sep(theta, z):
    """return dual sep in kpc given scalar angle sep in arcsec"""
    angle = (theta*u.arcsec).to(u.rad).value
    return (cosmo.angular_diameter_distance(z)*angle).to(u.kpc)


# cross-match catalogs

In [ ]:
# cross-match specifically for magellan
with open("/home/insepien/research-data/alpaka/alpaka_39fits.pkl","rb") as f:
    magel = pickle.load(f)

magel.reset_index(inplace=True)
magel.rename(columns={"Desig":"DESIG"},inplace=True)
magel_bm, _ = cross_bigmac(magel)

In [ ]:
# read in original sample
mul171 = read_mul171()
mul_bm, bigmac = cross_bigmac(mul171)

# get some stats on sub-kpc pairs for science justification
subkpc_mask = (bigmac['Primary System Type']=='Dual AGN Candidate') & (bigmac['Sep(kpc)']<1) & (bigmac['Sep(kpc)']>0)
subkpc_dual = bigmac[subkpc_mask]
dual_mask = (bigmac['Primary System Type']=='Dual AGN Candidate') & (bigmac['Sep(kpc)']>0)
dual = bigmac[dual_mask]
print("fraction of sub-kpc", len(subkpc_dual)/len(dual))
print("number of duals", len(dual))

# get some methods of measuring sub-kpc sep
anyl_meth = Counter(subkpc_dual['Parsed Analysis Method'])
print("1st most common method: ",anyl_meth.most_common(2)[0])
print("2nd most common method: ",anyl_meth.most_common(2)[1])

In [ ]:
esahubble = ESAHubble()
# get j1010 data 
j1010 = esahubble.query_criteria(proposal=14730)
j1010 = j1010.to_pandas()

# filter and HST search

In [ ]:
min_z = float(min(mul171['Z']))
max_z = float(max(mul171['Z']))
print(f"redshift range: {min_z:.3f}--{max_z:.3f}")
print(f"at z={min_z:.2f} wavelength: {np.floor((1+min_z)*4959)}--{np.ceil((1+min_z)*5007)}")
print(f"at z=0.2 wavelength: {np.floor((1.2)*4959)}--{np.ceil((1.2)*5007)}")
print(f"at z={max_z:.2f} wavelength: {np.floor((1+max_z)*4959)}--{np.ceil((1+max_z)*5007)}")
plt.hist((1+mul171['Z'])*5007,histtype='step',label="shifted 5007")
plt.hist((1+mul171['Z'])*4959,histtype='step',label="shifted 4959")
plt.xlabel("wavelength (angstroms)")
plt.ylabel("# of sources")
plt.title("shifted OIII lines in Mullaney full 171 sample");

In [ ]:
def search_hst(mul171_,filters_=["F621M","F689M"]):
    """search if sample has been observed with HST
        returns the search results, indices with search errors, and indices of objects that have been observed before"""
    res = []
    erred_ind = []
    for i in range(140,len(mul171_)):
        try:
            res.append(esahubble.cone_search_criteria(coordinates=pos(mul171_.loc[i]),
                                                radius=7*u.arcmin,
                                                instrument_name = ['WFC3'],
                                                filters = filters_))
        except:
            erred_ind.append(i)
    observed_ind = [i for i in range(len(res)) if len(res[i])!=0]
    return res, erred_ind, observed_ind

In [ ]:
# see what files is in an observation
esahubble.get_associated_files(observation_id="hst_12521_0e_wfc3_uvis_total")
# download a file
esahubble.download_file(file="hst_12521_0e_wfc3_uvis_total_drz.fits")